In [1]:
import pandas as pd

In [5]:
df_resume = pd.read_csv('./../resume_to_hackaton.csv', sep=';')
df_vac = pd.read_csv('./../vacancies_to_hackaton.csv', sep=';')
target_positions = ["Кладовщик",  "Комплектовщик", "Официант"]

In [37]:
df_resume.iloc[1]

uuid                         36c13102-4a62-477b-a475-f9a5b1c48218
position                                       Заведующий складом
organization                                        Оконный завод
description     руководство складом и цехом по производству ко...
start                                                     2014-02
end                                                       2015-09
Name: 1, dtype: object

In [40]:
len(df_resume[df_resume['position'] == "Кладовщик"])

11514

In [15]:
df_vac['description'][1]

'Обязанности:   Приготовление напитков в баре  Прием и подача заказов по столикам Работа с кассой  Требования:  Желание работать и зарабатывать. Опыт работы на подобной должности будет Вашим преимуществом. А вообще всему научим и всему обучим, так что не нужно бояться приходить без опыта работы). Наличие медицинской книжки будет Вашим преимуществом.  Условия:  График сменный 2\\2. Будние дни с 9-00 до 21-00. Выходные дни с 9-00 до 21-00. Оформление по ТК. Оплата: 100 руб/час + % + чаевые '

In [47]:
df_vac.iloc[5]

id                                                           17632940
name                                          Ночной контролер склада
area.name                                             Санкт-Петербург
city                                                   посёлок Шушары
company.id                                                     141100
company                                                            4Ф
company_link                            https://hh.ru/employer/141100
publication_date                             2016-07-05T10:20:51+0300
salary_from                                                     22000
salary_currency                                                   RUR
employment.name                                      Полная занятость
schedule.name                                          Сменный график
experience.name                                             Нет опыта
key_skills                                                        NaN
specializations     

In [24]:
df_vac["key_skills"][1].split(" | ")

['Ответственность',
 'Материальная ответственность',
 'Навыки межличностного общения',
 'Работа в команде',
 'Грамотная речь']

In [59]:
df_target_vac = df_vac[df_vac['name'].isin(["Кладовщик",  "Комплектовщик", "Официант"])]
df_target_resume = df_resume[df_resume['position'].isin(["Кладовщик",  "Комплектовщик", "Официант"])]